# Use RNN to classify pictures （MNIST）

### MNIST图片大小28*28，那么图片可以看作是长为28的序列，序列中的每个元素的特征维度是28.  
这样，将一张图片变成一个序列；考虑到RNN对previous信息的记忆功能，我们将图片从左往右输入网络的时候，网络可以记忆前面观察的东西。 虽然，将一张图片切割成了28份，但网络仍能记住前面的部分。

In [1]:
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import mnist
import numpy as np

In [2]:
# 定义超参数
batch_size = 100
learning_rate = 1e-3
num_epoches = 20

# 下载训练集 MNIST 手写数字训练集
train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
print(len(train_loader.dataset), len(test_loader.dataset))
train_dataset

60000 10000


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
# 定义 Recurrent Network 模型
class Rnn(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_layer, n_class):
        super(Rnn, self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(in_dim, hidden_dim, n_layer, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, n_class)

    def forward(self, x):
        # h0 = Variable(torch.zeros(self.n_layer, x.size(1),
        #   self.hidden_dim)).cuda()
        # c0 = Variable(torch.zeros(self.n_layer, x.size(1),
        #   self.hidden_dim)).cuda()
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.classifier(out)
        return out


In [5]:
model = Rnn(28, 128, 2, 10)  # 图片大小是28x28
use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
print(use_gpu)
if use_gpu:
    model = model.cuda()
# 定义loss和optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

True


In [6]:
import pixiedust

Pixiedust database opened successfully


In [7]:
# %%pixie_debugger
# 开始训练
for epoch in range(num_epoches):
    print('epoch {}'.format(epoch + 1))
    print('*' * 10)
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        b, c, h, w = img.size()
        assert c == 1, 'channel must be 1'
        img = img.squeeze(1)                     # ?squeeze(1)
        # img = img.view(b*h, w)
        # img = torch.transpose(img, 1, 0)
        # img = img.contiguous().view(w, b, -1)
        if use_gpu:
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img)
            label = Variable(label)
        # 向前传播
        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.data * label.size(0)  # loss.data[0]
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        running_acc += num_correct.data # [0]
        # 向后传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 300 == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(
            train_dataset))))
   


epoch 1
**********
[1/20] Loss: 0.801607, Acc: 0.725233
[1/20] Loss: 0.501206, Acc: 0.831850
Finish 1 epoch, Loss: 0.501206, Acc: 0.831850
epoch 2
**********
[2/20] Loss: 0.138276, Acc: 0.957300
[2/20] Loss: 0.119588, Acc: 0.963233
Finish 2 epoch, Loss: 0.119588, Acc: 0.963233
epoch 3
**********
[3/20] Loss: 0.082405, Acc: 0.973700
[3/20] Loss: 0.078249, Acc: 0.975800
Finish 3 epoch, Loss: 0.078249, Acc: 0.975800
epoch 4
**********
[4/20] Loss: 0.057571, Acc: 0.982333
[4/20] Loss: 0.060694, Acc: 0.981500
Finish 4 epoch, Loss: 0.060694, Acc: 0.981500
epoch 5
**********
[5/20] Loss: 0.051527, Acc: 0.985100
[5/20] Loss: 0.048425, Acc: 0.985667
Finish 5 epoch, Loss: 0.048425, Acc: 0.985667
epoch 6
**********
[6/20] Loss: 0.042926, Acc: 0.986133
[6/20] Loss: 0.040002, Acc: 0.987567
Finish 6 epoch, Loss: 0.040002, Acc: 0.987567
epoch 7
**********
[7/20] Loss: 0.034835, Acc: 0.989667
[7/20] Loss: 0.035165, Acc: 0.989417
Finish 7 epoch, Loss: 0.035165, Acc: 0.989417
epoch 8
**********
[8/20] L

In [10]:
# for epoch in range(num_epoches): 不用加循环，模型已经训练好了，所以，值是一样的
model.eval()
eval_loss = 0.
eval_acc = 0.
for data in test_loader:
    img, label = data
    b, c, h, w = img.size()
    assert c == 1, 'channel must be 1'
    img = img.squeeze(1)
    # img = img.view(b*h, w)
    # img = torch.transpose(img, 1, 0)
    # img = img.contiguous().view(w, b, h)
    if use_gpu:
        img = Variable(img, volatile=True).cuda()
        label = Variable(label, volatile=True).cuda()
    else:
        img = Variable(img, volatile=True)
        label = Variable(label, volatile=True)
    out = model(img)
    loss = criterion(out, label)
    eval_loss += loss.data * label.size(0)
    _, pred = torch.max(out, 1)
    num_correct = (pred == label).sum()
    eval_acc += num_correct.data  # [0]
print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
    test_dataset)), eval_acc / (len(test_dataset))))
print()

# 保存模型
torch.save(model.state_dict(), './rnn.pth')

/home/local/USHERBROOKE/shih3801/project_725/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app
/home/local/USHERBROOKE/shih3801/project_725/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  app.launch_new_instance()


Test Loss: 0.045104, Acc: 0.988700

